# Multiprocessing


## 1. 初始化共享資源
```python
manager = multiprocessing.Manager()
```
- 建立一個 Manager 物件。
- `multiprocessing.Manager()` 會啟動一個 伺服器進程 (server process)，它能夠管理 Python 物件，並讓其他子進程透過 proxy 代理 去共享和操作這些物件。

- Manager 支援的共享物件型別包括：

  - list, dict (共享容器)

  - Value, Array (共享變數)

  - Lock, RLock, Semaphore, Event (同步原語)
```
lock = manager.Lock()
```
- 呼叫 Manager 來建立一個 共享鎖。

- 這個 lock 不是單純的本地 threading.Lock，而是 跨進程的 proxy 鎖。

- 當多個進程想同時存取某段程式碼時，只會有一個進程能「獲得鎖」，其他進程必須等待。

- 用法：

```python
with lock:
    # 這段區域一次只能有一個進程執行
```
## 2. Lazy generator (攻擊招式隨機生成器)
```python
def lazy_return_random_attacks():
    attacks = {"kimura": "upper_body",
               "straight_ankle_lock":"lower_body",
               "arm_triangle":"upper_body",
               "keylock": "upper_body",
               "knee_bar": "lower_body"}

    while True:
        with lock:  # 確保同一時間只有一個 process 取招式
            random_attack = random.choice(list(attacks.keys()))

        yield random_attack

```
- `attacks` 是一個字典，存放格鬥技招式及攻擊部位。

- `while True` → 無限產生隨機招式。

- `with lock`: → 取得鎖，確保多個進程不會同時呼叫 random.choice 造成混亂。

- `yield` → 使函數變成 生成器 (generator)，可以逐次懶加載返回攻擊招式。


## 3. 每個進程的執行函數
```python
def print_attacks(num):
    attacks = lazy_return_random_attacks()
    for i in range(num):
        attack = next(attacks)
        print(f"Process {os.getpid()}: {attack}\n")
```

- 參數 `num` → 指定要印出多少次攻擊招式。

- `os.getpid()` → 顯示目前進程的 PID，方便觀察多進程輸出差異。

## 4. 建立進程池並分配任務
```python
pool = multiprocessing.Pool(processes=2)
pool.map(print_attacks, [3]*2)
```

- `multiprocessing.Pool(processes=2)` → 建立一個 含 2 個子進程的進程池。

- `pool.map(print_attacks, [3]*2)` → 讓兩個進程都呼叫 print_attacks(3)，各印出 3 次攻擊招式。


## 5. 執行流程

- 主程式建立進程池。

- 兩個子進程分別執行 print_attacks(3)。

- 每個子進程各自建立一個 generator，並在 loop 中呼叫 next() 取得隨機攻擊。

- lock 確保同一時間只有一個進程能抽招式，避免 race condition。

In [6]:
import multiprocessing
import random
import os

# Create proxy lock that works in multiprocessing
manager = multiprocessing.Manager()
lock = manager.Lock()

# Generator function to lazily yield items
def lazy_return_random_attacks():

    attacks = {"kimura": "upper_body",
               "straight_ankle_lock":"lower_body",
               "arm_triangle":"upper_body",
               "keylock": "upper_body",
               "knee_bar": "lower_body"}

    while True:
        # Use lock to prevent concurrent access
        # to shared state from different processes
        with lock:
            random_attack = random.choice(list(attacks.keys()))

        yield random_attack

# Function to be executed concurrently
def print_attacks(num):

    attacks = lazy_return_random_attacks()

    for i in range(num):
        attack = next(attacks)
        print(f"Process {os.getpid()}: {attack}\n")

# Create process pool
pool = multiprocessing.Pool(processes=10)
pool.map(print_attacks, [10]*5)

Process 7261: straight_ankle_lock
Process 7259: arm_triangle

Process 7263: knee_bar

Process 7265: knee_bar
Process 7261: straight_ankle_lock

Process 7259: keylock


Process 7267: arm_triangle

Process 7265: kimura


Process 7263: kimura
Process 7259: knee_bar
Process 7267: straight_ankle_lock
Process 7265: arm_triangle

Process 7261: kimura


Process 7263: knee_bar


Process 7261: keylock
Process 7267: arm_triangle

Process 7259: straight_ankle_lock
Process 7265: knee_bar
Process 7263: straight_ankle_lock




Process 7261: arm_triangle
Process 7265: straight_ankle_lock

Process 7267: knee_bar
Process 7259: arm_triangle
Process 7263: straight_ankle_lock




Process 7265: arm_triangle
Process 7261: keylock
Process 7267: knee_bar



Process 7263: knee_bar
Process 7259: keylock


Process 7265: keylock
Process 7261: knee_bar


Process 7267: kimura
Process 7265: arm_triangle
Process 7263: kimura


Process 7259: straight_ankle_lock



Process 7267: kimura
Process 7261: kimura
Process 7265:

[None, None, None, None, None]

# Threading

這段程式示範 多執行緒 (threading)，並用 Lock 來避免同時操作共享資源造成錯誤。

逐段說明
1. 建立全域鎖
```python
lock = threading.Lock()
```

建立一把全域的鎖，讓多個執行緒在進入「臨界區」時必須先拿到鎖。

作用：保證同一時間只有一個執行緒能執行鎖內的程式碼。

In [4]:
import threading
import random

# Global lock to synchronize thread access to shared generator
lock = threading.Lock()

# To store threads
threads = []

# Generator lazily provides random items from a list
def lazy_return_random_attacks():

    # Collection of items to randomly yield
    attacks = {"kimura": "upper_body",
               "straight_ankle_lock":"lower_body",
               "arm_triangle":"upper_body",
               "keylock": "upper_body",
               "knee_bar": "lower_body"}

    while True:
        # Use lock to prevent concurrent access to shared state
        with lock:
            random_attack = random.choice(list(attacks.keys()))

        # Yield next random item
        yield random_attack

# Function to consume random items from generator
def thread_function(thread_id):

    # Get generator
    attacks = lazy_return_random_attacks()

    # Print out 10 random items
    for i in range(10):
        attack = next(attacks)
        print(f"Thread {thread_id}: {attack}")

# Create and start 5 threads to demonstrate concurrence
for i in range(5):
    thread = threading.Thread(target=thread_function, args=(i,))
    threads.append(thread)
    thread.start()

# Wait for threads to complete
for thread in threads:
    thread.join()

Thread 0: arm_triangle
Thread 0: kimura
Thread 0: arm_triangle
Thread 0: kimura
Thread 0: kimura
Thread 0: kimura
Thread 0: arm_triangle
Thread 0: straight_ankle_lock
Thread 0: kimura
Thread 0: kimura
Thread 1: straight_ankle_lock
Thread 1: straight_ankle_lock
Thread 1: straight_ankle_lock
Thread 1: kimura
Thread 1: kimura
Thread 1: keylock
Thread 1: arm_triangle
Thread 1: keylock
Thread 1: keylock
Thread 1: kimura
Thread 2: arm_triangle
Thread 2: straight_ankle_lock
Thread 2: kimura
Thread 2: kimura
Thread 2: kimura
Thread 2: arm_triangle
Thread 2: knee_bar
Thread 2: keylock
Thread 2: straight_ankle_lock
Thread 2: straight_ankle_lock
Thread 3: kimura
Thread 3: kimura
Thread 3: keylock
Thread 3: arm_triangle
Thread 3: kimura
Thread 3: straight_ankle_lock
Thread 3: arm_triangle
Thread 3: straight_ankle_lock
Thread 3: knee_bar
Thread 3: keylock
Thread 4: kimura
Thread 4: straight_ankle_lock
Thread 4: straight_ankle_lock
Thread 4: knee_bar
Thread 4: arm_triangle
Thread 4: knee_bar
Thread 